In [1]:
import nltk
from nltk.tokenize import sent_tokenize
import pandas as pd

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vitor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
conectivos = [
    "para concluir",
    "em resumo",
    "em suma",
    "portanto",
    "assim",
    "logo",
    "por conseguinte",
    "concluindo",
    "em conclusão",
    "em consequência",
    "deste modo",
    "por este motivo",
    "desse modo",
    "dessa forma",
    "consequentemente",
    "dessarte",
    "destarte",
    "em vista disso",
    "em vista do exposto",
    "em razão disso",
    "em virtude disso",
    "em decorrência disso",
    "em síntese",
    "em poucas palavras",
    "para finalizar",
    "por fim",
    "em última análise",
    "finalizando",
    "assim sendo",
    "dessa maneira",
    "dessa sorte",
    "dessa ordem de ideias",
    "em decorrência",
    "por essa razão",
    "por esse motivo",
    "de modo que",
    "de forma que",
    "enfim",
    "para sintetizar",
    "para resumir",
    "para encerrar",
    "diante disso",
    "tendo em vista",
    "considerando o exposto",
    "posto isso",
    "isso posto",
    "com base nisso",
    "conclui-se que",
    "conclui-se",
    "é possível concluir que",
    "é necessário concluir que",
    "a partir disso",
    "como resultado",
    "tudo considerado",
    "à guisa de conclusão",
    "urge que",
    "espera-se que",
    "sendo assim",
    "dessa maneira",
    "à luz do exposto",
    "à vista disso",
    "em face do exposto",
    "em virtude do exposto",
    "torna-se evidente que",
    "torna-se claro que",
    "porém",
    "é necessário",
    "urge",
    "diante dos argumentos",
]

In [4]:
def extrair_trecho_final(texto, conectivos, percentFim=0.70, min_len=100, max_len=1000, minPercent=0.08):
    """
    - texto (str): texto completo da redação.
    - conectivos (list[str]): lista de conectivos conclusivos esperados (minúsculos).
    - percentFim (float): porcentagem a partir da qual começa o trecho final (ultimos: 70%).
    - min_len (int): comprimento mínimo aceitável para o trecho extraído.
    - max_len (int): comprimento máximo aceitável para o trecho extraído.
    - minPercent (float): percentual mínimo que o trecho deve representar em relação ao texto total.
    """
    trecho = texto[int(len(texto) * percentFim):].strip()
    trecho_lower = trecho.lower()
    min_pct = len(texto) * minPercent

    def valido(txt): return min_len < len(txt) < max_len and len(txt) >= min_pct

    # Busca conectivo logo após um ponto, exclamação ou interrogação
    for i, char in enumerate(trecho):
        if char in '.!?':
            txt = trecho[i+1:].lstrip()
            for conj in conectivos:
                if ' '.join(txt.split()[:len(conj.split())]).lower() == conj and valido(txt): #quebra conj para ver o tamanho
                    print(f"[1] Conectivo após ponto: {conj}")
                    return txt.strip()

    # Procura conectivos em todo o trecho e classifica por maiúsculo ou minúsculo
    maius, minus = [], []
    for conj in conectivos:
        pos = trecho_lower.find(conj)
        if pos != -1:
            orig = trecho[pos:pos+len(conj)]
            if orig[0].isupper():
                maius.append((pos, conj))
            else:
                minus.append((pos, conj))
    maius.sort(), minus.sort()

    # Se houver conectivo com inicial maiúscula, retorna a penúltima ou última ocorrência
    if maius:
        if len(maius) >= 2:
            pos, conj = maius[-2]
        else:
            pos, conj = maius[-1]
        txt = trecho[pos:]
        if valido(txt):
            print(f"[2] Conectivo Maiúsculo encontrado: {conj}")
            return txt.strip()

    # tenta após o primeiro ponto, se não encontrou conectivo válido
    ponto = trecho.find('.')
    if ponto != -1:
        txt = trecho[ponto+1:].lstrip()
        if valido(txt):
            print("[3] ponto usado.")
            return txt

    # Se houver conectivo minúsculo, retorna a penúltima ou última ocorrência
    if minus:
        if len(minus) >= 2:
            pos, conj = minus[-2]
        else:
            pos, conj = minus[-1]
        txt = trecho[pos:]
        if valido(txt):
            print(f"[4] Conectivo minúsculo encontrado: {conj}")
            return txt.strip()

    # tenta após a vírgula
    virg = trecho.find(',')
    if virg != -1:
        txt = trecho[virg+1:].lstrip()
        if valido(txt):
            print("[5] vírgula usada.")
            return txt
        
    # Nada encontrado: retorna o trecho final como está
    print("[6] Nenhum conectivo encontrado, retornando trecho completo.")
    return trecho.strip()

In [5]:
bd = pd.read_csv("teste val - Página1.csv")

In [6]:
redacoes = bd["Redação"]

In [7]:
def preencher_tabela(redacoes):
    dados = []

    for i, texto in enumerate(redacoes):
        texto = texto.strip()
        conclusao = extrair_trecho_final(texto, conectivos)
        QtdeCaracteres = len(conclusao)
        qtde_frases = len(sent_tokenize(conclusao, language="portuguese"))
        porcentagem = round((len(conclusao) / len(texto)) * 100, 2)

        dados.append(
            {
                "Redação": texto,
                "QtdeCaracteres": QtdeCaracteres,
                "Intervencao": conclusao,
                "QtdeSentencasIntervencao": qtde_frases,
                "PorcentagemCaracteresIntervencao": porcentagem,
            }
        )

    return pd.DataFrame(dados)

In [8]:
dados = preencher_tabela(redacoes)

[2] Conectivo Maiúsculo encontrado: por fim
[2] Conectivo Maiúsculo encontrado: sendo assim
[2] Conectivo Maiúsculo encontrado: portanto
[2] Conectivo Maiúsculo encontrado: portanto
[2] Conectivo Maiúsculo encontrado: dessa forma
[3] ponto usado.
[1] Conectivo após ponto: diante dos argumentos
[3] ponto usado.
[1] Conectivo após ponto: diante dos argumentos
[2] Conectivo Maiúsculo encontrado: em conclusão
[2] Conectivo Maiúsculo encontrado: por conseguinte
[3] ponto usado.
[2] Conectivo Maiúsculo encontrado: em síntese
[2] Conectivo Maiúsculo encontrado: assim
[2] Conectivo Maiúsculo encontrado: portanto
[2] Conectivo Maiúsculo encontrado: desse modo
[2] Conectivo Maiúsculo encontrado: em suma
[3] ponto usado.
[2] Conectivo Maiúsculo encontrado: portanto
[2] Conectivo Maiúsculo encontrado: deste modo
[3] ponto usado.


In [9]:
nome_arquivo = f"ValidacaoRes.xlsx"
dados.to_excel(nome_arquivo, index=False)

__________________________________

In [10]:
from difflib import SequenceMatcher

def similaridade(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [11]:
dfT = pd.read_csv("teste val - Página1 (1).csv")

In [12]:
dfT["similaridade"] = dfT.apply(
    lambda row: similaridade(str(row["Intervencao"]), str(row["Conclusão"])), axis=1
)

# Filtra os casos onde a similaridade é boa (por exemplo, acima de 0.80)
dfT_bons = dfT[(dfT["similaridade"] >= 0.80) | (dfT["QtdeCaracteres"] <= 80)]

print(f"Número de casos parecidos: {len(dfT_bons)} de {len(dfT)}")
acuracia = len(dfT_bons) / len(dfT) * 100
print(f"Acuracia de: {acuracia:.2f}%")

Número de casos parecidos: 20 de 21
Acuracia de: 95.24%


In [13]:
dfT["similaridade"] = dfT.apply(
    lambda row: similaridade(str(row["Intervencao"]), str(row["Conclusão"])), axis=1
)

# Filtra os casos onde a similaridade é boa (por exemplo, acima de 0.80)
dfT_bons = dfT[(dfT["similaridade"] <= 0.80) & (dfT["similaridade"] >= 0.50)]

print(f"Número de casos parecidos: {len(dfT_bons)} de {len(dfT)}")
acuracia = len(dfT_bons) / len(dfT) * 100
print(f"Acuracia de: {acuracia:.2f}%")

Número de casos parecidos: 1 de 21
Acuracia de: 4.76%


In [14]:
dfT["similaridade"] = dfT.apply(
    lambda row: similaridade(str(row["Intervencao"]), str(row["Conclusão"])), axis=1
)

# Filtra os casos onde a similaridade é boa (por exemplo, acima de 0.80)
dfT_bons = dfT[dfT["similaridade"] <= 0.50]

print(f"Número de casos parecidos: {len(dfT_bons)} de {len(dfT)}")
acuracia = len(dfT_bons) / len(dfT) * 100
print(f"Acuracia de: {acuracia:.2f}%")

Número de casos parecidos: 0 de 21
Acuracia de: 0.00%
